# Doctor Finder Multi-Agent System



In [1]:
from kaggle_secrets import UserSecretsClient
import os
GOOGLE_API_KEY = UserSecretsClient().get_secret('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
print('API key loaded')

API key loaded


In [2]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search, FunctionTool
from google.genai import types

retry_config = types.HttpRetryOptions(
    attempts=5, exp_base=7, initial_delay=1,
    http_status_codes=[429,500,503,504]
)
print('ADK imported')

ADK imported


## Session Initializer — FINAL VERSION

In [3]:
session_init_agent = Agent(
    name='SessionInitializer',
    model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    instruction='''
Extract symptoms and location from the user message.
Output ONLY in this exact format:
symptoms: <extracted symptoms>
location: <extracted location>
''',
    output_key='patient_info'
)
print('SessionInitializer ready')

SessionInitializer ready


## Parallel Specialty Guess Agents

## Aggregator

In [4]:
guess_internal = Agent(
    name='InternalMedGuess', model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    instruction="Based on patient info {patient_info}, predict the most likely medical specialty.", output_key='guess_internal')

guess_emergency = Agent(
    name='EmergencyGuess', model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    instruction="Based on patient info {patient_info}, predict the most likely medical specialty.", output_key='guess_emergency')

guess_clinical = Agent(
    name='ClinicalGuess', model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    instruction="Based on patient info {patient_info}, predict the most likely medical specialty.", output_key='guess_clinical')

parallel_specialty = ParallelAgent(
    name='SpecialtyParallel', sub_agents=[guess_internal, guess_emergency, guess_clinical])

print('Parallel ready')

Parallel ready


In [5]:
specialty_aggregator = Agent(
    name='SpecialtyAggregator', model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    instruction='Choose best specialty among {guess_internal}, {guess_emergency}, {guess_clinical}.',
    output_key='final_specialty')
print('Aggregator ready')

Aggregator ready


## Doctor Search Agent

## Explanation Agent

In [6]:
doctor_search_agent = Agent(
    name='DoctorSearchAgent', model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    instruction="""Patient info: {patient_info}. 
Use google_search tool to find doctors specializing in {final_specialty}. 
Extract location from patient_info.

Search for doctors with COMPLETE information including:
- Full name
- Specific clinic/hospital address
- Phone number or contact details
- Patient ratings/reviews if available
- Consultation fees if available

Perform thorough search to gather all available details.""",
    tools=[google_search], output_key='doctor_results')
print('Doctor search ready')

Doctor search ready


In [7]:
explanation_agent = Agent(
    name='ExplanationAgent', 
    model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    instruction='''Format the results as a well-organized markdown table:

**Patient Information:**
{patient_info}
Recommended Specialty: {final_specialty}

**Recommended Doctors:**
Create a table with columns: Doctor Name | Specialty | Location | Contact | Rating | Fee
Use the search results from: {doctor_results}

CRITICAL FILTERING RULES:
1. ONLY include doctors who have AT LEAST 3 of these details:
   - Specific clinic/hospital address (not just city name)
   - Phone number or contact information
   - Patient rating/reviews
   - Consultation fee

2. DO NOT include doctors with:
   - Only a name and city
   - Missing contact information
   - Hospital name as location without specific address

3. Show the most complete profiles first (doctors with 4+ details at top)

4. If a detail is truly not available after filtering, use "N/A"

5. If no doctors meet the criteria, display: "No doctors found with complete information. Please try a broader search or different location."

Format each doctor's information clearly with all available details.''',
    output_key='summary'
)
print('Explanation ready')

Explanation ready


## Loop Refinement

In [8]:
def exit_loop():
    return {'summary': 'Approved summary', 'status': 'approved'}

exit_tool = FunctionTool(exit_loop)

critic = Agent(
    name='Critic', model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    instruction='Review summary: {summary}. APPROVED or feedback.',
    output_key='critique')

refiner = Agent(
    name='Refiner', model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    instruction='If {critique} == APPROVED, call exit_loop. Otherwise refine {summary}.',
    tools=[exit_tool], output_key='summary')

loop = LoopAgent(name='RefinementLoop', sub_agents=[critic, refiner], max_iterations=1)
print('Loop ready')

Loop ready


## Final Pipeline

In [9]:
pipeline = SequentialAgent(
    name='DoctorFinderPipeline',
    sub_agents=[
        session_init_agent,
        parallel_specialty,
        specialty_aggregator,
        doctor_search_agent,
        explanation_agent,
        loop
    ]
)

runner = InMemoryRunner(agent=pipeline)
print('Pipeline and runner ready')

Pipeline and runner ready


## Run the System

In [10]:
# Interactive input for symptoms and location
symptoms = input("Please describe your symptoms: ")
location = input("Please enter your location (city, country): ")

# Construct the query
user_query = f"symptoms: {symptoms}; location: {location}"
print(f"\n🔍 Searching for doctors based on:\n{user_query}\n")

Please describe your symptoms:  neck pain
Please enter your location (city, country):  islamabad,pakistan



🔍 Searching for doctors based on:
symptoms: neck pain; location: islamabad,pakistan



In [11]:
# Run the doctor finder system with user input
response = await runner.run_debug(user_query)

# Extract and display the final summary
final_summary = None
for event in response:
    if hasattr(event, 'author') and event.author == 'ExplanationAgent':
        if hasattr(event, 'actions') and event.actions and event.actions.state_delta:
            final_summary = event.actions.state_delta.get('summary')
            break

# Display the formatted result
if final_summary:
    from IPython.display import Markdown, display
    print("\n" + "="*80)
    print("📋 DOCTOR FINDER RESULTS")
    print("="*80 + "\n")
    display(Markdown(final_summary))
else:
    print("No summary found in response")


 ### Created new session: debug_session_id

User > symptoms: neck pain; location: islamabad,pakistan
SessionInitializer > symptoms: neck pain
location: islamabad,pakistan
ClinicalGuess > Given the symptom of neck pain and the location of Islamabad, Pakistan, the most likely medical specialty to consult would be **Orthopedics** or **Neurology**.

*   **Orthopedics** specializes in the musculoskeletal system, which includes the bones, joints, ligaments, tendons, and muscles of the neck. Many causes of neck pain, such as muscle strain, herniated discs, or degenerative joint disease, fall under this specialty.
*   **Neurology** deals with disorders of the nervous system, including the brain, spinal cord, and nerves. If the neck pain is suspected to be related to nerve compression, pinched nerves, or other neurological conditions, a neurologist would be the appropriate specialist.

In some cases, a **Rheumatologist** might also be considered if autoimmune conditions causing neck pain are s


📋 DOCTOR FINDER RESULTS



**Patient Information:**
symptoms: neck pain
location: islamabad,pakistan
Recommended Specialty: **Orthopedics** or **Neurology**

**Recommended Doctors:**

| Doctor Name                   | Specialty                                          | Location                                        | Contact | Rating                                           | Fee                       |
| :---------------------------- | :------------------------------------------------- | :---------------------------------------------- | :------ | :----------------------------------------------- | :------------------------ |
| Dr. Shabir Awan               | Orthopedic and Minimally Invasive Spinal Surgeon   | Islamabad Specialist Clinic (ISC), F-8 Islamabad  | N/A     | 100% for professionalism, 98% for compassionate care | PKR 1,200 to PKR 4,000    |
| Dr. Wazahat Taqvi             | Orthopedic and Spine Surgeon                       | Islamabad Specialist Clinic (ISC), F-8 Islamabad  | N/A     | N/A                                              | PKR 1,200 to PKR 4,000    |
| Dr. Abdul Basit               | Orthopedic and Spine Surgeon                       | Islamabad Specialist Clinic (ISC), F-8 Islamabad  | N/A     | N/A                                              | PKR 1,200 to PKR 4,000    |
| Dr. M. Sarfaraz               | Orthopedic Surgeon                                 | N/A                                             | N/A     | N/A                                              | PKR 1,200 to PKR 4,000    |
| Prof. Dr. Rao Suhail Yasin Khan | Neurologist                                        | N/A                                             | N/A     | N/A                                              | PKR 2,500 to PKR 5,000, or PKR 500 to PKR 3,000, or PKR 300 to PKR 3,000 |
| Dr. Soban Sarwar Gondal       | Neurologist (also specializes in Pain Medicine and Neuro Surgery) | N/A                                             | N/A     | N/A                                              | N/A                       |
| Dr. Riaz Mahmood              | Neurologist                                        | N/A                                             | N/A     | N/A                                              | N/A                       |
| Dr. Muhammad Babar Khan       | Neurologist                                        | N/A                                             | N/A     | N/A                                              | N/A                       |
| Dr. Faridullah Shah           | Neurologist                                        | N/A                                             | N/A     | N/A                                              | N/A                       |
| Dr. Saeed Arif                | Neurologist                                        | N/A                                             | N/A     | N/A                                              | N/A                       |
| Dr. Ahmad                     | Neurologist                                        | N/A                                             | N/A     | N/A                                              | N/A                       |
| Dr. Husnain Hashim            | Neurologist                                        | Neuro Care Center                               | N/A     | N/A                                              | N/A                       |
| Dr. Neelma Khattak            | Neurologist                                        | N/A                                             | N/A     | N/A                                              | N/A                       |
| Asst. Prof. Dr. Samina Majeed | Neurologist                                        | N/A                                             | N/A     | N/A                                              | N/A                       |
| Dr. Mujahid Khan              | Neurologist                                        | N/A                                             | N/A     | N/A                                              | N/A                       |
| Dr. Zahid Rustam              | Neurologist                                        | N/A                                             | N/A     | N/A                                              | N/A                       |
| Muhammad Farhan Farhat        | Orthopedic Surgeon                                 | General Practice Healthcare Center              | N/A     | N/A                                              | N/A                       |
| Dr. Arif Kaleem               | Orthopedic Surgeon                                 | Elite Healthcare                                | N/A     | N/A                                              | N/A                       |
| Dr. Sohail Rehman             | Orthopedic Surgeon                                 | Saeed International Hospital                    | N/A     | N/A                                              | N/A                       |
| Dr. Asad Ullah Burki          | Orthopedic Surgeon                                 | Premium International Hospital                  | N/A     | N/A                                              | N/A                       |